In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import norm 
import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import fiona
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import seaborn as sns
plt.style.use('seaborn')
sns.set(style="white")
pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [2]:
import pickle

# scaler_file = open('scaler.pickle','rb')
# final_scaler = pickle.load(scaler_file)
model_file = open('model.pickle','rb')
final_model = pickle.load(model_file)
# scaler_file.close()
model_file.close()

In [3]:
ht= pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [5]:
def zero_val_bed_bath(row):
    """
    Checking extreame number of rooms in the house
    """
    if row['bedrooms'] == 0:
        row['bedrooms'] = row['floors']
    if row['bathrooms'] < 1:
        row['bathrooms'] = 1
    if row['bedrooms'] > 10 :
        row['bedrooms'] = 10
    return row

In [6]:
ht = ht.apply(zero_val_bed_bath, axis = 1)

In [7]:
ht.drop(columns = [ 'id', 'view', 'sqft_living15', 'sqft_lot15'], inplace = True)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [8]:
ht['yr_ren'] = np.nan

In [9]:
def yr_ren(row):
    
    if row['yr_renovated'] == 0:
        row['yr_ren'] = 0
    if row['yr_renovated'] != 0:
        row['yr_ren'] = 2021 - row['yr_built']
        
    return row 

In [10]:
ht = ht.apply(yr_ren, axis = 1)

In [11]:
ht['yr_updated'] = np.nan

In [12]:
def yr_update(row):
    
    if row['yr_renovated'] == 0:
        row['yr_updated'] = 2021 - row['yr_built']
    if row['yr_renovated'] != 0:
        row['yr_updated'] = 2021 - row['yr_built']
        
    return row 

In [13]:
ht = ht.apply(yr_update, axis = 1)

In [14]:
ht['percent_bedbath'] = np.nan
ht['has_golden_ratio'] = np.nan

In [15]:
def cal_ratio_range(row):
    golden_ratio = (2/3)
    golden_ratio_plus = golden_ratio + (golden_ratio * .10)
    golden_ratio_minus = golden_ratio - (golden_ratio * .10)
    
    if row['percent_bedbath'] <= golden_ratio_plus and row['percent_bedbath'] >= golden_ratio_minus:
            row['has_golden_ratio'] = 1
    else:
        row['has_golden_ratio'] = 0
    
    return row

In [16]:
ht = ht.apply(cal_ratio_range, axis = 1)

In [17]:
# Based off of bathrooms pros and housetipster 

def ratio_bed_bath(row):
    
    ratio_bed_bath = row['bathrooms'] / row['bedrooms']
    golden_ratio = (2/3)
    row['percent_bedbath'] = abs(golden_ratio - ratio_bed_bath) 
    
    return row

In [18]:
ht = ht.apply(ratio_bed_bath, axis = 1)

In [19]:
ht['ratio_liv_lot'] = np.nan

In [20]:
def ratio_living_lot(row):
    
    row['ratio_liv_lot'] = row['sqft_lot'] / row['sqft_living']
    return row

In [21]:
ht = ht.apply(ratio_living_lot, axis = 1)

In [22]:
ht = pd.concat([ht, pd.get_dummies(ht['zipcode'])], 1)

In [23]:
ht = ht.drop(columns = 'zipcode')

In [24]:
ht = pd.concat([ht, pd.get_dummies(ht['grade'])], 1)

In [25]:
ht = ht.drop(columns = 'grade')

In [26]:
# ht.columns = ht.columns.astype(str)

In [27]:
ht

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,yr_ren,yr_updated,percent_bedbath,has_golden_ratio,ratio_liv_lot,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,1,4,5,6,7,8,9,10,11,12,13
0,20140827T000000,4.0,2.50,2270,11500,1.0,0,3,1540,730,1967,0,47.7089,-122.241,0,54,0.041667,0,5.066079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,20150218T000000,4.0,2.50,2270,11500,1.0,0,3,1540,730,1967,0,47.7089,-122.241,0,54,0.041667,0,5.066079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,20141107T000000,3.0,2.50,1470,1779,2.0,0,3,1160,310,2005,0,47.5472,-121.998,0,16,0.166667,0,1.210204,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,20141203T000000,3.0,1.75,1280,16200,1.0,0,3,1030,250,1976,0,47.7427,-122.071,0,45,0.083333,0,12.656250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,20150115T000000,4.0,2.75,2830,8126,2.0,0,3,2830,0,2005,0,47.4863,-122.140,0,16,0.020833,0,2.871378,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,20140521T000000,3.0,2.50,1530,1131,3.0,0,3,1530,0,2009,0,47.6993,-122.346,0,12,0.166667,0,0.739216,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4319,20150223T000000,4.0,2.50,2310,5813,2.0,0,3,2310,0,2014,0,47.5107,-122.362,0,7,0.041667,0,2.516450,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4320,20140623T000000,2.0,1.00,1020,1350,2.0,0,3,1020,0,2009,0,47.5944,-122.299,0,12,0.166667,0,1.323529,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4321,20150116T000000,3.0,2.50,1600,2388,2.0,0,3,1600,0,2004,0,47.5345,-122.069,0,17,0.166667,0,1.492500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [28]:
ht = ht.rename(columns={1: 'lowest_g', 4:'dnmc', 5:'Poor', 6:'bare_min', 
                   7:'average', 8:'above_avg', 9:'good', 10:'high_qua', 11:'higher_qua',
                   12:'excellent_qua', 13:'mansion' })

In [29]:
fts_for_poly = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_basement', 'yr_ren', 'yr_updated']

In [30]:
def poly_df (df, degree):
    
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_data = poly.fit_transform(df)
    poly_columns = poly.get_feature_names(df.columns)
    df_poly = pd.DataFrame(poly_data, columns=poly_columns)
    return df_poly

In [31]:
poly_df = poly_df(ht[fts_for_poly], 2)

In [32]:
not_poly_fts = [x for x in ht.columns if x not in poly_df]

In [33]:
ht = pd.merge(poly_df, ht[not_poly_fts], left_index=True, right_index=True)

In [34]:
ht = pd.concat([ht, pd.get_dummies(ht['waterfront'])], 1)

In [35]:
ht = ht.rename(columns={0: "No_Waterfront", 1: "Waterfront"})

In [36]:
ht['water_sqft_lot'] = np.nan

In [37]:
def water_lot(row):
    if row['waterfront'] == 1:
        row['water_sqft_lot'] = row['Waterfront'] * row['sqft_lot'] 
    if row['waterfront'] == 0:
        row['water_sqft_lot'] = 0
    return row

In [38]:
ht = ht.apply(water_lot, axis=1)

In [39]:
ht = ht.drop(columns='waterfront')

In [40]:

ht = ht.drop(columns= 'date')
ht = ht.drop(columns= 'lat')
ht = ht.drop(columns= 'long')

In [41]:
effect_feat = ['bedrooms', 'bathrooms', 
            'sqft_living', 'sqft_lot', 'floors', 'condition', 'sqft_above', 
            'sqft_basement', 'yr_built', 'yr_renovated', 
            'percent_bedbath','has_golden_ratio', 'lowest_g','low_g', 'dnmc', 'Poor', 
            'bare_min', 'average', 'above_avg', 'good', 'high_qua', 'higher_qua', 
            'excellent_qua', 'mansion', 'No_Waterfront',  'Waterfront', 'water_sqft_lot',
            '98001', '98002', '98003', '98004', '98005', '98006',
           '98007', '98008', '98010', '98011', '98014', '98019', '98022', '98023',
           '98024', '98027', '98028', '98029', '98030', '98031', '98032', '98033',
           '98034', '98038', '98039', '98040', '98042', '98045', '98052', '98053',
           '98055', '98056', '98058', '98059', '98065', '98070', '98072', '98074',
           '98075', '98077', '98092', '98102', '98103', '98105', '98106', '98107',
           '98108', '98109', '98112', '98115', '98116', '98117',
           '98118', '98119', '98122', '98125', '98126', '98133', '98136', '98144',
           '98146', '98148', '98155', '98166', '98168', '98177', '98178', '98188',
           '98198', '98199']

In [42]:
ht.columns = ht.columns.astype(str)

In [43]:
ht[effect_feat].shape

(4323, 87)

In [44]:
# transformed_holdout = final_scaler.transform(ht)

## Step 3: Predict the holdout set

In [45]:
final_answers = final_model.predict(ht[effect_feat])

In [46]:
final_answers

array([573809.20867698, 573809.20867698, 423584.53138506, ...,
       348545.5787214 , 398344.41701372, 349545.07529812])

In [47]:
df = pd.DataFrame(final_answers)

## Step 4: Export your predictions

In [48]:
df.to_csv('housing_preds_Rafael_ferreira.csv')